### OpenStreetMap Project
### Data Wrangling with MongoDB
##### map area:Sydney,Austrilia

### 1.地图数据中遇到的问题
为了方便的查看地图数据中可能存在的问题，我们先获取约3Mb左右的悉尼地区的样本数据进行观测，利用“案例分析：OpenStreetMap 数据”课程中探索数据的方法，我们先将此部分样本数据转换成json数据，并导入到MongoDB中，方便进行数据探索。这里我们发现两个问题需要进行探索
1. 数据中存在"St."类似的街道简写。
2. 数据中存在错误的post codes

#### 1.1 查看street names

In [ ]:
#获取样本数据里所有街道名称
data = db.sydneysample.aggregate([
    {"$match":{"address.street":{"$exists":1 }}},
    {"$group" : {"_id" : "$address.street",
         "count" : {"$sum" : 1}}},
    {"$sort" : {"count" : -1}}])

In [ ]:
{u'count': 51, u'_id': u'George Street'}
{u'count': 18, u'_id': u'Clarence Street'}
{u'count': 17, u'_id': u'Martin Place'}
{u'count': 16, u'_id': u'Kent Street'}
{u'count': 12, u'_id': u'York Street'}
{u'count': 10, u'_id': u'Pitt Street'}
{u'count': 8, u'_id': u'Sussex Street'}
{u'count': 7, u'_id': u'Erskine Street'}
{u'count': 6, u'_id': u'Macquarie Street'}
{u'count': 6, u'_id': u'King Street'}
{u'count': 5, u'_id': u"O'Connell Street"}
{u'count': 5, u'_id': u'Bridge Street'}
{u'count': 4, u'_id': u'Hunter Street'}
{u'count': 3, u'_id': u'Margaret Street'}
{u'count': 3, u'_id': u'Castlereagh Street'}
{u'count': 3, u'_id': u'Macquarie Place'}
{u'count': 3, u'_id': u'Barrack Street'}
{u'count': 3, u'_id': u'Harrington Street'}
{u'count': 2, u'_id': u'Bond Street'}
{u'count': 2, u'_id': u'Bulletin Place'}
{u'count': 2, u'_id': u'Elizabeth Street'}
{u'count': 2, u'_id': u'Argyle Street'}
{u'count': 2, u'_id': u'Clarance Street'}
{u'count': 2, u'_id': u'Lime Street'}
{u'count': 1, u'_id': u'Hickson Road'}
{u'count': 1, u'_id': u'Market Street'}
{u'count': 1, u'_id': u'Shelley Street'}
{u'count': 1, u'_id': u'Conservatorium Road'}
{u'count': 1, u'_id': u'Bennelong Point'}
{u'count': 1, u'_id': u'Chifley Square'}
{u'count': 1, u'_id': u'Bent Street'}
{u'count': 1, u'_id': u'Barangaroo Avenue'}
{u'count': 1, u'_id': u'Phillip St.'}
{u'count': 1, u'_id': u'Farrer Place'}
{u'count': 1, u'_id': u'Margaret St'}
{u'count': 1, u'_id': u'George street'}
{u'count': 1, u'_id': u'Playfair St & Argyle Street, The Rocks NSW 2000\u6fb3\u6d32'}
{u'count': 1, u'_id': u'Carrington Street'}
{u'count': 1, u'_id': u'Cnr / Playfair & Argyle St'}
{u'count': 1, u'_id': u'Alfred Street'}
{u'count': 1, u'_id': u'Palings Lane'}
{u'count': 1, u'_id': u'The Promanade'}
{u'count': 1, u'_id': u'Bligh Street'}
{u'count': 1, u'_id': u'Philip Street'}
{u'count': 1, u'_id': u'Abercrombie Lane'}
{u'count': 1, u'_id': u'Phillip Street'}
{u'count': 1, u'_id': u'King Street Offramp'}

样本数据中共出现47个街道名称，从中可以看到出现"Phillip St.""Margaret St""Cnr / Playfair & Argyle St"等简写，我利用之前课程里的方法，将这些简写字符串更新为完整的街道名称，如"Phillip St."更新为"Phillip Street"。

#### 1.2 查看post codes

In [ ]:
#获取样本数据里所有postal codes
data = db.sydneysample.aggregate([
{"$match":{"address.postcode":{"$exists":1 }}},
{"$group" : {"_id" : "$address.postcode",
     "count" : {"$sum" : 1}}},
{"$sort" : {"count" : -1}}])

In [ ]:
{u'count': 87, u'_id': u'2000'}
{u'count': 6, u'_id': u'NSW 2000'}

根据查询澳大利亚的邮编可以知道,2000属于New South Wales的邮编范围，NSW则为New South Wales其简写，这里我把邮编里包含非数字的字符串进行截取，更新成标准邮编格式，即"NSW 2000"更新为"2000"。

#### 1.3 数据清洗
##### 1.3.1 清洗Street name

In [1]:
#对街道名简写进行处理
"""
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "G:\A_Udacity_data_analyst\course\project3\sydney_sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# 街道简称记全称映射关系
mapping = { "St": "Street",
            "St.": "Street",
            "Rd":"Road",
            "Rd.":"Road",
            "Ave":"Avenue",
            "Ave.":"Avenue"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):

    # 更新街道名称
    for item in name.split():
        if item in mapping:
            name = name.replace(item,mapping[item])
        else:
            pass
    return name


def test():
    st_types = audit(OSMFILE)
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name




if __name__ == '__main__':
    test()

King Street Offramp => King Street Offramp
Bennelong Point => Bennelong Point
Phillip St. => Phillip Street
Playfair St & Argyle Street, The Rocks NSW 2000澳洲 => Playfair Street & Argyle Streetreet, The Rocks NSW 2000澳洲
Cnr / Playfair & Argyle St => Cnr / Playfair & Argyle Street
Margaret St => Margaret Street
George street => George street


##### 1.3.2 清洗 postcodes
这里采用mongodb update方法进行邮编处理

In [ ]:
data = db.sydneysample.update(
{"address.postcode" : "NSW 2000"},
{"$set" : {"address.postcode" : "2000"}},
multi = True
)

### 2.数据探索
#### 文件大小
sydney_australia.osm ..........308MB  
sydney_australia.osm.json ....341MB

In [ ]:
# 数据总数
> db.sydney.find().count()
1597325

# 唯一用户总数
> db.sydney.distinct("created.user")
2070

# 节点总数
> db.sydney.find({"type":"node"}).count()
1404655

# 路径总数
> db.sydney.find({"type":"way"}).count()
192489

# 商店数量
> db.sydney.find({"shop":{"$exists":1}}).count()
3057

# 用户贡献排名
> db.sydney.aggregate([{"$match":{"created.user":{"$exists":1}}},
                                {"$group":{"_id":"$created.user",
                                "count":{"$sum":1}}}, {"$sort":{"count": -1}}, {"$limit":10}])

{u'count': 117436, u'_id': u'balcoath'}
{u'count': 91304, u'_id': u'inas'}
{u'count': 71882, u'_id': u'TheSwavu'}
{u'count': 61057, u'_id': u'ChopStiR'}
{u'count': 53188, u'_id': u'aharvey'}
{u'count': 47942, u'_id': u'Leon K'}
{u'count': 44706, u'_id': u'cleary'}
{u'count': 40613, u'_id': u'Rhubarb'}
{u'count': 37979, u'_id': u'AntBurnett'}
{u'count': 36149, u'_id': u'Warin61'}

### 3.关于数据集的思考
观察数据发现，相当多的数据created.version为1，表名该数据仅创建时编辑了一次，后续再无修改，一般来说为了保证地理数据的准确性，是需要对地理位置信息进行更新的。这里我们分别针对node和way统计其version情况:

In [14]:
# 获取node数据的 version情况
import pandas as pd

def get_db():
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client.cities
    return db

if __name__ == "__main__":
    # For local use
    db = get_db()


    data = db.sydney.aggregate([{"$match":{"type":"node"}},
                                {"$group":{"_id":"$created.version",
                                "count":{"$sum":1}}}, {"$sort":{"count": -1}}])

    df = pd.DataFrame(list(data))
    df["percent"] = df["count"]/sum(df["count"])*100
    print df


   _id    count    percent
0    1  1066476  75.924408
1    2   179285  12.763632
2    3    66994   4.769427
3    4    42898   3.053988
4    5    23363   1.663255
5    6    11362   0.808882
6    7     5461   0.388779
7    8     3236   0.230377
8    9     1809   0.128786
9   10     1118   0.079592
10  11      743   0.052896
11  12      553   0.039369
12  13      373   0.026555
13  14      255   0.018154
14  15      169   0.012031
15  16      138   0.009824
16  17      104   0.007404
17  18       60   0.004272
18  19       55   0.003916
19  20       41   0.002919
20  21       26   0.001851
21  23       25   0.001780
22  27       21   0.001495
23  22       19   0.001353
24  25       16   0.001139
25  24       11   0.000783
26  26       11   0.000783
27  28        9   0.000641
28  35        5   0.000356
29  29        5   0.000356
30  34        3   0.000214
31  30        3   0.000214
32  32        2   0.000142
33  33        2   0.000142
34  36        1   0.000071
35  31        1   0.000071
3

In [15]:
# 获取way数据的 version情况
import pandas as pd

def get_db():
    # For local use
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client.cities
    return db

if __name__ == "__main__":
    # For local use
    db = get_db()


    data = db.sydney.aggregate([{"$match":{"type":"way"}},
                                {"$group":{"_id":"$created.version",
                                "count":{"$sum":1}}}, {"$sort":{"count": -1}}])

    df = pd.DataFrame(list(data))
    df["percent"] = df["count"]/sum(df["count"])*100
    print df


    _id  count    percent
0     1  76984  39.993974
1     2  38229  19.860356
2     3  25739  13.371673
3     4  13498   7.012349
4     5   7818   4.061531
5     6   6585   3.420975
6     7   5688   2.954974
7     8   4643   2.412086
8     9   3564   1.851534
9    10   2572   1.336180
10   11   1849   0.960574
11   12   1295   0.672766
12   13    974   0.506003
13   14    670   0.348072
14   15    516   0.268067
15   16    359   0.186504
16   17    284   0.147541
17   18    254   0.131956
18   19    182   0.094551
19   20    153   0.079485
20   21    122   0.063380
21   22     90   0.046756
22   23     76   0.039483
23   24     69   0.035846
24   26     52   0.027015
25   25     45   0.023378
26   28     28   0.014546
27   27     22   0.011429
28   29     20   0.010390
29   31     17   0.008832
30   30     17   0.008832
31   33     10   0.005195
32   32     10   0.005195
33   35      9   0.004676
34   34      7   0.003637
35   37      6   0.003117
36   38      5   0.002598
37   40     

可以看到node数据中，version为1的数据占了75.92%，而version在3次以内的数据约占92%，，绝大部分数据的更新次数不超过3次，甚至相当一部分的数据自从创建之后再无更新。
    
在way数据中，version次数显著增加，绝大部分数据的更新次数集中在10次以内，说明way的变化较node来说，更新会更频繁一点。不过总体来说，OpenStreetMap上的数据更新频率还是比较低的，建议可以采取激励的方式鼓励用户更新相关数据，提升数据的准确度。  
  
鼓励用户尽量频繁的更新数据，可以有效提升数据的准确性以及时效性，在应用数据时会更具备参考性。但同时也要考虑，更多更频繁的更新也会带来一些问题，不同用户之间对同一位置的修改也可能导致数据的不一致性。但总的来说，数据的频繁更新会更好提升数据准确度和有效性，为使用数据的用户带来更好体验。
   
另外，在进行数据处理时，根据用户贡献度可以看出，贡献程度大的用户很可能是电脑或者智能设备（短时间内上传大量数据），也可能是一些忠实用户，这些用户在提供数据的准确性以及专业度比较高。而一些贡献很小的用户，很可能对osm的熟悉度不高，导致上传的数据的格式可能存在问题较多，且与其他用户重复的几率也比较大，这样我们在清洗数据时，可以考虑过滤一些贡献较少的用户创建的数据，以提升数据准确度。
  
当然可以预见的问题是，筛选掉这些数据后，可能会存在部分位置信息的缺失，但相对来说，这里面可能会存在一些质量不高的数据，过滤之后更易于我们进行数据清洗，所以综合来看，还是利大于弊的。

### 4.额外的数据探索

In [ ]:
# 最多的美食
db.sydney.aggregate([{"$match":{"amenity":{"$exists":1},
        "amenity":"restaurant"}}, {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
        {"$sort":{"count": -1}}, {"$limit":5}])

In [ ]:
{u'_id': None, u'count': 471}
{u'_id': u'thai', u'count': 66}
{u'_id': u'chinese', u'count': 63}
{u'_id': u'italian', u'count': 52}
{u'_id': u'pizza', u'count': 40}

比较出乎意料，该地区最多的美食居然是泰国菜，然后中国菜紧跟其后，再然后是意大利美食以及大家都爱的披萨。

In [ ]:
# 最多的商店类型
db.sydney.aggregate([{"$match":{"shop":{"$exists":1}}}, {"$group":{"_id":"$shop", "count":{"$sum":1}}},
        {"$sort":{"count": -1}}, {"$limit":5}])

In [ ]:
{u'_id': u'supermarket', u'count': 458}
{u'_id': u'convenience', u'count': 331}
{u'_id': u'clothes', u'count': 176}
{u'_id': u'mall', u'count': 171}
{u'_id': u'alcohol', u'count': 138}

超市占据了商店分类中的榜首，接着是便利店，服装店的数量排在了第三名，鉴于地图选取范围基本为悉尼市区，这个结果也能理解。另外两个出现较多的商店类型是购物中心和酒吧，看来悉尼地区的商业氛围还是非常浓重的。

### 5.总结这种
通过对悉尼地区的地图数据的清洗整理与探索，我了解到了OpenStreetMap这种共享式开放街区数据的优势，这种地图类的wiki项目，以非常快捷且便利的方式获取了大量丰富的地图数据，但是由于参与编辑数据的人数众多，导致部分数据格式并不十分规范，若需要更好的利用这些数据，必须针对数据进行清洗整理，并设计一个比较好的清洗方法进行调用，以供后续针对这些地图数据进行更有趣的挖掘。   
这次数据清洗整理的过程，不但让我了解了整体数据获取、整理、清洗、探索、分析的数据处理过程，同时也了解了一些关于悉尼这种城市的一些有趣信息，希望下次在规划悉尼之行时，可以利用osm的数据，来更详尽的补充旅行计划，哈哈！！